<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/File_Utilities_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# %%writefile FileUtilities.py
import glob, os, pathlib, sys
from os.path import *
from pathlib import Path

class ColorOutput:
    LITE = '\033[96m'; HEADER = '\033[95m'; BLUE = '\033[94m';
    WARNING = '\033[93m'; GREEN = '\033[92m'; RED = '\033[91m'; 
    DARK = '\033[90m'; UNDERLINE = '\033[4m';BOLD = '\033[1m'; WHITE = '\033[0m'

global C
C = ColorOutput()

class ColorUtil():
    def colorOutput():
        pass

class AllFileUtilities(ColorUtil):
    def __init__(self):
        self.C = ColorUtil.colorOutput()
        self.homePath = Path.home()
        self.cwdPath = Path.cwd()
        self.allFiles = glob.glob('**', recursive=True)
        self.allFilesDict = {}

    def listAllFiles(self):
        tempDict = {}
        fileCount = 0
        print(f'{C.BLUE}{self.cwdPath}')
        for fil in self.allFiles:
            fullPath = Path(realpath(fil)) # POSIX path
            realPath = realpath(fullPath) # String path
            if isdir(fullPath) and not realPath.__contains__('__pycache__'):  
                print(f'\n{C.BLUE}{fullPath}')
            elif isfile(fullPath) and not realPath.__contains__('__pycache__'): 
                fileCount += 1
                tempDict[fileCount] = fullPath
                print(f'{C.WHITE}{fileCount}. {fullPath.name}')
        self.allFilesDict = tempDict
            

afu = AllFileUtilities()
afu.listAllFiles()

/content
1. FileUtilities.py

/content/sample_data
2. README.md
3. anscombe.json
4. mnist_train_small.csv
5. mnist_test.csv
6. california_housing_train.csv
7. california_housing_test.csv
